In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Membaca dataset dengan pemisah tanda semicolon (;)
dataset = pd.read_csv('alamatkntr.csv', sep=';')

# Memeriksa struktur dataset
print(dataset.head())  # Menampilkan beberapa baris pertama dataset
print(dataset.info())  # Menampilkan informasi dataset


# Menangani nilai yang hilang
dataset = dataset.dropna()  # Menghapus baris data yang mengandung nilai yang hilang

# Menyimpan dataset yang telah diproses
dataset.to_csv('percobaan1.csv', index=False)

  singer_id    Latitude    Longitude  Ratings
0   09i8o76  -6.200.567  106.780.635      4.3
1   73r5t02  -6.178.944  106.792.172      3.8
2   12q9w47  -6.217.353  106.828.801      4.7
3   51v2b78  -6.229.472  106.836.617      4.5
4   38n7m54  -6.203.402  106.846.361      4.1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   singer_id  159 non-null    object 
 1   Latitude   150 non-null    object 
 2   Longitude  150 non-null    object 
 3   Ratings    150 non-null    float64
dtypes: float64(1), object(3)
memory usage: 5.1+ KB
None


In [2]:
# Pisahkan fitur (features) dan target (target) dari dataset
X = dataset[['Latitude', 'Longitude']]  # Features
y = dataset['Ratings']  # Target

# Bagi dataset menjadi data pelatihan dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Periksa ukuran data pelatihan dan data uji
print("Jumlah data pelatihan:", len(X_train))
print("Jumlah data uji:", len(X_test))

Jumlah data pelatihan: 120
Jumlah data uji: 30


In [3]:
import pandas as pd
from math import radians
from sklearn.metrics.pairwise import haversine_distances

# Load the dataset
dataset = pd.read_csv('percobaan1.csv')

# Preprocess the dataset
dataset['Latitude'] = dataset['Latitude'].str.replace('.', '').astype(float) / 1000000
dataset['Longitude'] = dataset['Longitude'].str.replace('.', '').astype(float) / 1000000

# Specify the user's latitude and longitude
user_latitude = radians(-6.200567)  # Replace with the actual user's latitude
user_longitude = radians(106.780635)  # Replace with the actual user's longitude

# Calculate the distances between the user and each office location
distances = haversine_distances(
    [[user_latitude, user_longitude]],
    dataset[['Latitude', 'Longitude']]
)

# Convert the distances to kilometers
distances_km = distances.flatten() * 6371.0

# Add the distances to the dataset
dataset['Distance'] = distances_km

# Sort the dataset by distance in ascending order
dataset = dataset.sort_values(by='Distance')

# Recommend the nearest office to the user
nearest_office = dataset.iloc[0]
recommended_office = {
    'Singer ID': nearest_office['singer_id'],
    'Latitude': nearest_office['Latitude'],
    'Longitude': nearest_office['Longitude'],
    'Distance (km)': nearest_office['Distance']
}

# Print the recommended office
print("Recommended Office:")
print(recommended_office)

Recommended Office:
{'Singer ID': '29g2d31', 'Latitude': -0.062566, 'Longitude': 1.069937, 'Distance (km)': 5045.382485568197}


<ipython-input-3-0b45d2a36055>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Latitude'] = dataset['Latitude'].str.replace('.', '').astype(float) / 1000000
<ipython-input-3-0b45d2a36055>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Longitude'] = dataset['Longitude'].str.replace('.', '').astype(float) / 1000000


In [4]:
# Sort the dataset by distance in ascending order
dataset = dataset.sort_values(by='Distance')

# Recommend the nearest office to the user
nearest_office = dataset.iloc[0]
recommended_office = {
    'Singer ID': nearest_office['singer_id'],
    'Latitude': nearest_office['Latitude'],
    'Longitude': nearest_office['Longitude'],
    'Distance (km)': nearest_office['Distance']
}

# Print the recommended office
print("Recommended Office:")
print(recommended_office)


Recommended Office:
{'Singer ID': '29g2d31', 'Latitude': -0.062566, 'Longitude': 1.069937, 'Distance (km)': 5045.382485568197}


In [5]:
# Calculate the average rating of the nearest offices
average_rating = dataset['Ratings'].mean()

# Print the average rating
print("Average Rating:", average_rating)

# Check if the average rating meets the threshold
if average_rating >= 4.0:
    print("Recommended office meets the rating threshold.")
else:
    print("Recommended office does not meet the rating threshold.")


Average Rating: 4.312666666666667
Recommended office meets the rating threshold.


In [6]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# Load the dataset
dataset = pd.read_csv('percobaan1.csv')

# Preprocess the dataset
dataset['Latitude'] = dataset['Latitude'].str.replace('.', '').astype(float) / 1000000
dataset['Longitude'] = dataset['Longitude'].str.replace('.', '').astype(float) / 1000000
dataset['Ratings'] = dataset['Ratings'].fillna(0)

# Specify the user's latitude and longitude
user_latitude = radians(-6.200567)  # Replace with the actual user's latitude
user_longitude = radians(106.780635)  # Replace with the actual user's longitude

# Calculate the distances between the user and each office location
distances = []
for _, row in dataset.iterrows():
    office_latitude = radians(row['Latitude'])
    office_longitude = radians(row['Longitude'])

    # Haversine formula
    d_latitude = office_latitude - user_latitude
    d_longitude = office_longitude - user_longitude
    a = sin(d_latitude / 2) ** 2 + cos(user_latitude) * cos(office_latitude) * sin(d_longitude / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Earth's radius in kilometers
    distances.append(distance)

# Add the distances to the dataset
dataset['Distance'] = distances

# Set the maximum distance threshold
max_distance = 10  # Replace with your desired maximum distance in kilometers

# Filter the offices within the maximum distance
filtered_dataset = dataset[dataset['Distance'] <= max_distance]

# Sort the filtered dataset by distance in ascending order
sorted_dataset = filtered_dataset.sort_values(by='Distance')

# Check if any offices are available within the maximum distance
if len(sorted_dataset) > 0:
    # Recommend the nearest office to the user
    nearest_office = sorted_dataset.iloc[0]
    recommended_office = {
        'Singer ID': nearest_office['singer_id'],
        'Latitude': nearest_office['Latitude'],
        'Longitude': nearest_office['Longitude'],
        'Distance (km)': nearest_office['Distance'],
        'Ratings': nearest_office['Ratings']
    }

    # Print the recommended office
    print("Recommended Office:")
    print(recommended_office)
else:
    print("No offices available within the maximum distance.")

# Calculate the average rating of the filtered dataset
average_rating = filtered_dataset['Ratings'].mean()
print("Average Rating:", average_rating)



Recommended Office:
{'Singer ID': '09i8o76', 'Latitude': -6.200567, 'Longitude': 106.780635, 'Distance (km)': 0.0, 'Ratings': 4.3}
Average Rating: 4.29


<ipython-input-6-d334cffc8d4c>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Latitude'] = dataset['Latitude'].str.replace('.', '').astype(float) / 1000000
<ipython-input-6-d334cffc8d4c>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Longitude'] = dataset['Longitude'].str.replace('.', '').astype(float) / 1000000


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.16897558839773927


In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model KNN
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.19124000000000008


In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Decision Tree
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.16853342592592607


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.16492051503980945


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Gradient Boosting
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.15535202495480005


In [13]:
pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.9
    Uninstalling flax-0.6.9:
      Successfully uninstalled flax-0.6.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
xar

In [16]:
import tensorflow as tf
import tensorflowjs as tfjs

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Pisahkan fitur dan label
X = dataset[['Distance']]
y = dataset['Ratings']

# Inisialisasi model Sequential
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(1,)))
model.add(Dense(1))

# Kompilasi model
model.compile(optimizer='adam', loss='mean_squared_error')

# Latih model
model.fit(X, y, epochs=10, batch_size=32)

# Simpan model Keras ke dalam format HDF5
model.save('model.h5')


Epoch 1/10
5/5 [==============================] - 1s 4ms/step - loss: 2085189.3750
Epoch 2/10
5/5 [==============================] - 0s 3ms/step - loss: 1296631.0000
Epoch 3/10
5/5 [==============================] - 0s 3ms/step - loss: 745448.9375
Epoch 4/10
5/5 [==============================] - 0s 3ms/step - loss: 349414.3125
Epoch 5/10
5/5 [==============================] - 0s 3ms/step - loss: 131203.7656
Epoch 6/10
5/5 [==============================] - 0s 3ms/step - loss: 28545.5938
Epoch 7/10
5/5 [==============================] - 0s 3ms/step - loss: 1700.7961
Epoch 8/10
5/5 [==============================] - 0s 3ms/step - loss: 4654.7773
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 13252.3760
Epoch 10/10
5/5 [==============================] - 0s 3ms/step - loss: 15410.8936


In [18]:
# Konversi model Keras ke format TensorFlow.js
tfjs.converters.save_keras_model(model, 'tfjs_model')

In [46]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('model.h5')

In [73]:
import pandas as pd

# Load the new data
new_data = pd.read_csv('Data_kantor(L).csv')

# Memeriksa struktur dataset
print(dataset.head())  # Menampilkan beberapa baris pertama dataset
print(dataset.info())  # Menampilkan informasi dataset


  singer_id  Latitude   Longitude  Ratings  Distance
0   09i8o76 -6.200567  106.780635      4.3  0.000000
1   73r5t02 -6.178944  106.792172      3.8  2.721685
2   12q9w47 -6.217353  106.828801      4.7  5.642083
3   51v2b78 -6.229472  106.836617      4.5  6.973218
4   38n7m54 -6.203402  106.846361      4.1  7.272459
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   singer_id  150 non-null    object 
 1   Latitude   150 non-null    float64
 2   Longitude  150 non-null    float64
 3   Ratings    150 non-null    float64
 4   Distance   150 non-null    float64
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None


In [74]:
print(new_data.columns)

Index(['Latitude;Longitude;Ratings'], dtype='object')


In [76]:
new_data[['Latitude', 'Longitude', 'Ratings']] = new_data['Latitude;Longitude;Ratings'].str.split(';', expand=True)



In [77]:
new_data['Latitude'] = new_data['Latitude'].apply(lambda x: str(x).replace('.', ''))
new_data['Longitude'] = new_data['Longitude'].apply(lambda x: str(x).replace('.', ''))

new_data['Latitude'] = new_data['Latitude'].astype(float)
new_data['Longitude'] = new_data['Longitude'].astype(float)

new_data['Ratings'] = new_data['Ratings'].apply(lambda x: x if x.isnumeric() else 0)
new_data['Ratings'] = new_data['Ratings'].astype(float)

predictions = model.predict(new_data)


ValueError: ignored

In [69]:
new_data['Latitude'] = new_data['Latitude'].apply(lambda x: str(x).replace('.', ''))
new_data['Longitude'] = new_data['Longitude'].apply(lambda x: str(x).replace('.', ''))

new_data['Latitude'] = new_data['Latitude'].astype(float)
new_data['Longitude'] = new_data['Longitude'].astype(float)
new_data['Rating'] = new_data['Rating'].astype(float)

predictions = model.predict(new_data)
print(predictions)


ValueError: ignored

In [78]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# Membaca data baru
new_data = pd.read_csv('Data_kantor(L).csv')

# Preprocessing data baru
X_new = new_data[['Distance']].values  # Sesuaikan dengan fitur yang digunakan dalam model
# Lakukan preprocessing tambahan jika diperlukan, seperti normalisasi atau transformasi lainnya

# Load model yang telah Anda simpan sebelumnya
model = load_model('model.h5')

# Melakukan prediksi pada data baru
predictions = model.predict(X_new)

# Menampilkan hasil prediksi
for i, pred in enumerate(predictions):
    print('Data Baru ke-', i+1, ':', pred[0])  # Adjust this based on your output format


KeyError: ignored